In [53]:
import cv2
import numpy as np
import time
import math
from tracker import *

# create tracker object
tracker = EuclideanDistTracker()

# capturing or reading video
# cap = cv2.VideoCapture('./Car_data/video.avi')
cap = cv2.VideoCapture('/Users/analeegraig/Documents/Computer Vision/final_project_data/cars.mp4')

count_line_position = 600

# minimum contour width
min_contour_width = 40 

# minimum contour height
min_contour_height = 40

# define variables to calculate speed
fps_start_time = time.time()
fps = 0
total_frames = 0
speed_kph = 0
start_point = (0,0)
pixels_per_meter = 10

# define function get centroid
def get_centroid(x, y, w, h):
    x1 = int(w / 2)
    y1 = int(h / 2)
    cx = x + x1
    cy = y + y1
    return cx,cy

# define function to calculate speed
def calculate_speed(start_point, end_point, fps):
    distance_pixels = abs(end_point[0] - start_point[0])
    distance_meters = distance_pixels / pixels_per_meter  # Adjust 
    speed_mps = distance_meters / fps
    speed_kph = speed_mps * 3.6
    return speed_kph

# define function to identify transform frame
def transform_frame(frame):
    grey = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(grey, (3, 3), 5)
    img_sub = algo.apply(blur)
    dilat = cv2.dilate(img_sub, np.ones((5, 5)))
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    dilated = cv2.morphologyEx(dilat, cv2.MORPH_CLOSE, kernel, iterations = 2)    
    return dilated

# create background subtractor
algo = cv2.createBackgroundSubtractorMOG2(history = 200, varThreshold = 200)

In [54]:
offset = 6
count = 0

while True:
    # get frame
    ret, frame1 = cap.read()
    count += 1
    #roi = frame1[300:1100, 300:]
    roi = frame1[200:500, 300:]
    
    if not ret:
        break
    
    # 1: Object detection
    ## transform frame
    dilatada = transform_frame(roi)
    
    ## find contours
    contourShape, h = cv2.findContours(dilatada, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    detections = []
    for (i,c) in enumerate(contourShape):
        (x, y, w, h) = cv2.boundingRect(c)
        
        ## check if contour is large enough
        contour_valid = (w >= min_contour_width) and (h >= min_contour_height)
        if not contour_valid:
            continue
        
        # if countour large enough, continue
        ## save boundary box coordinates
        detections.append([x, y, w, h])
        
        ## draw centroid
        centroid = get_centroid(x, y, w, h)
        cv2.circle(roi, centroid, 4, (0, 0, 255), -1)    
    
    # 2: Object Tracking
    boxes_ids = tracker.update(detections)
    for box_id in boxes_ids:
        x, y, w, h, id = box_id
        cv2.putText(roi, "V" + str(id), (x, y - 10), 0, 1, (255,0,0), 3)
        cv2.rectangle(roi, (x,y), (x+w,y+h), (0, 255, 0), 2)
        
    cv2.imshow('original', roi)
    
    # define keyboard break
    key = cv2.waitKey(60)
    if key == ord('q'):
        break
    
cv2.destroyAllWindows()
cap.release()

{1: (652, 262)}
{1: (639, 252)}
{1: (628, 233)}
{1: (614, 228), 2: (650, 269)}
{2: (647, 247), 1: (614, 228)}
{2: (647, 247), 1: (604, 208)}
{2: (647, 247), 1: (613, 197), 3: (402, 271)}
{3: (412, 260), 1: (613, 197)}
{3: (412, 260), 1: (606, 179)}
{3: (416, 249), 1: (606, 179)}
{3: (416, 249), 1: (596, 164)}
{3: (412, 239), 1: (596, 164)}
{3: (412, 239), 1: (582, 149)}
{3: (414, 230), 1: (582, 149)}
{3: (399, 244), 1: (582, 149)}
{3: (399, 244), 1: (569, 133)}
{3: (399, 244), 1: (577, 122), 4: (415, 214)}
{4: (415, 194), 1: (577, 122)}
{4: (415, 194), 1: (570, 106)}
{4: (405, 175), 1: (570, 106)}
{4: (405, 175), 1: (560, 95)}
{4: (406, 161), 1: (560, 95)}
{4: (406, 161), 1: (558, 84)}
{4: (407, 147), 1: (558, 84)}
{4: (407, 147), 1: (554, 74)}
{4: (407, 133), 1: (554, 74)}
{4: (407, 133), 1: (551, 64)}
{4: (399, 120), 1: (551, 64)}
{4: (399, 120), 1: (543, 55)}
{4: (398, 106), 1: (543, 55)}
{4: (398, 106), 1: (539, 46)}
{4: (397, 94), 1: (539, 46)}
{4: (397, 94), 1: (535, 37)}
{4: (39

{21: (216, 156), 18: (522, 34)}
{21: (216, 156), 18: (519, 29)}
{21: (223, 137), 18: (519, 29)}
{21: (223, 137), 18: (512, 25)}
{21: (226, 120), 18: (512, 25)}
{21: (226, 120), 18: (509, 20)}
{21: (232, 104), 18: (509, 20)}
{21: (237, 89)}
{21: (243, 76)}
{21: (248, 62)}
{21: (251, 52)}
{21: (255, 38)}
{21: (258, 32)}
{21: (266, 27)}
{21: (266, 21)}
{22: (733, 267), 23: (178, 271)}
{23: (199, 258), 22: (733, 267)}
{23: (199, 258), 22: (728, 256)}
{23: (199, 258), 22: (723, 246)}
{23: (205, 246), 22: (723, 246)}
{22: (711, 236), 23: (205, 246)}
{22: (711, 236), 23: (212, 235)}
{22: (694, 218), 23: (212, 235)}
{22: (694, 218), 23: (222, 225)}
{22: (684, 198), 23: (222, 225)}
{22: (684, 198), 23: (232, 205)}
{22: (676, 180), 23: (232, 205)}
{22: (676, 180), 23: (237, 182)}
{22: (668, 163), 23: (237, 182)}
{22: (668, 163), 23: (246, 162)}
{22: (658, 148), 23: (246, 162)}
{22: (658, 148), 23: (253, 142)}
{22: (647, 132), 23: (253, 142)}
{22: (647, 132), 23: (256, 125)}
{22: (637, 119), 23: 

{36: (571, 124)}
{36: (566, 110)}
{36: (559, 97)}
{36: (553, 86)}
{36: (546, 75)}
{36: (544, 63), 37: (890, 272)}
{37: (881, 262), 36: (544, 63)}
{37: (881, 262), 36: (536, 54)}
{37: (873, 252), 36: (536, 54)}
{37: (873, 252), 36: (532, 43)}
{37: (865, 244), 36: (532, 43)}
{37: (865, 244), 36: (529, 38)}
{37: (851, 236), 36: (529, 38)}
{37: (851, 236), 36: (521, 33)}
{37: (832, 223), 36: (521, 33)}
{37: (832, 223), 36: (519, 28)}
{37: (832, 223), 36: (516, 24), 38: (811, 204)}
{38: (796, 187), 36: (516, 24)}
{38: (796, 187), 36: (511, 20)}
{38: (779, 171), 36: (511, 20)}
{38: (765, 155)}
{38: (754, 142)}
{38: (742, 128)}
{38: (727, 116)}
{38: (716, 106)}
{38: (706, 94)}
{38: (695, 84)}
{38: (687, 74)}
{38: (680, 65)}
{38: (671, 56)}
{38: (660, 48)}
{38: (652, 38)}
{38: (645, 34)}
{38: (639, 30)}
{38: (630, 25)}
{38: (627, 22), 39: (182, 266)}
{39: (190, 254), 38: (627, 22)}
{39: (202, 242)}
{39: (210, 232)}
{39: (215, 209)}
{39: (225, 187)}
{39: (230, 165)}
{39: (235, 146)}
{39: (238, 

{60: (413, 157), 59: (535, 60), 58: (266, 46)}
{60: (413, 157), 59: (530, 50), 58: (266, 46)}
{60: (413, 157), 59: (530, 50), 58: (268, 37)}
{60: (411, 140), 59: (530, 50), 58: (268, 37)}
{60: (411, 140), 59: (525, 41), 58: (268, 37)}
{60: (411, 140), 59: (525, 41), 58: (270, 32)}
{60: (411, 124), 59: (525, 41), 58: (270, 32)}
{60: (411, 124), 59: (521, 35), 58: (270, 32)}
{60: (411, 124), 59: (521, 35), 58: (272, 27)}
{60: (409, 109), 59: (521, 35), 58: (272, 27)}
{60: (409, 109), 59: (518, 30), 58: (272, 27)}
{60: (409, 109), 59: (518, 30), 58: (276, 23)}
{60: (407, 95), 59: (518, 30), 58: (276, 23)}
{60: (407, 95), 59: (516, 25), 58: (276, 23)}
{60: (407, 82), 59: (516, 25)}
{60: (407, 82), 59: (508, 21)}
{60: (404, 69), 59: (508, 21)}
{60: (401, 58), 61: (111, 279)}
{61: (132, 267), 60: (401, 58)}
{61: (132, 267), 60: (401, 52)}
{61: (146, 255), 60: (401, 52)}
{61: (146, 255), 60: (404, 45)}
{61: (155, 244), 60: (404, 45)}
{61: (155, 244), 60: (401, 39)}
{61: (163, 234), 60: (401, 

TypeError: 'NoneType' object is not subscriptable

# unedited code

In [ ]:
offset = 6
count = 0

while True:
    # get frame
    ret, frame1 = cap.read()
    count += 1
    
    if not ret:
        break
        
    # transform frame
    dilatada = transform_frame(frame1)
    
    # find contours
    contourShape, h = cv2.findContours(dilatada, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    # object tracking
    for (i,c) in enumerate(contourShape):
        (x, y, w, h) = cv2.boundingRect(c)
        
        # check if contour is large enough
        contour_valid = (w >= min_contour_width) and (h >= min_contour_height)
        if not contour_valid:
            continue
        
        # if countour large enough, continue
        # draw boundary box
        cv2.rectangle(frame1, (x,y),(x+w,y+h), (255, 0, 0), 2)        
        
        # draw centroid
        centroid = get_centroid(x, y, w, h)
        cv2.circle(frame1,centroid, 4, (0, 0, 255), -1)    
    
    cv2.imshow('original', frame1)
    
    # define keyboard break
    key = cv2.waitKey(1)
    if key == ord('q'):
        break
    
cv2.destroyAllWindows()
cap.release()

# old code

In [ ]:
offset = 6
count = 0
center_points_prev_frame = []

tracking_objects = {}
track_id = 0

while True:
    # get frame
    ret, frame1 = cap.read()
    count += 1
    
    if not ret:
        break
    
    # transform frame
    dilatada = transform_frame(frame1)
    
    # find contours
    contourShape, h = cv2.findContours(dilatada, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    # center points
    center_points_cur_frame = []
    
    # object tracking
    for (i,c) in enumerate(contourShape):
        (x, y, w, h) = cv2.boundingRect(c)
        
        # check if contour is large enough
        contour_valid = (w >= min_contour_width) and (h >= min_contour_height)
        if not contour_valid:
            continue
        
        # if countour large enough, continue
        # draw boundary box
        cv2.rectangle(frame1, (x,y),(x+w,y+h), (255, 0, 0), 2)        
        
        # draw centroid
        centroid = get_centroid(x, y, w, h)
        center_points_cur_frame.append(centroid)
        
        # cv2.circle(frame1,centroid, 4, (0, 0, 255), -1)
        
        for pt in center_points_cur_frame:
            for pt2 in center_points_prev_frame:
                distance = math.hypot(pt2[0] - pt[0], pt2[1] - pt[1])

                if distance < 20:
                    tracking_objects[track_id] = pt
                    track_id += 1

        print('TRACKING OBJECTS')
        print(tracking_objects)
                
                
        #cv2.circle(frame1, pt, 4, (0, 0, 255), -1)
    
    """
    cv2.imshow('original', frame1)
    
    # make copy of current points
    center_points_prev_frame = center_points_cur_frame.copy()
    
    # define keyboard break
    key = cv2.waitKey(1)
    if key == ord('q'):
        break
    
cv2.destroyAllWindows()
cap.release()
"""

In [ ]:
center_points

# old code

In [ ]:
# create background subtractor
algo = cv2.createBackgroundSubtractorMOG2()

detect = []
offset = 6
counter = 0
center_points_prev_frame = []

tracking_objects = {}
track_id = 0

# define variables to calculate speed
fps_start_time = time.time()
fps = 0
total_frames = 0
speed_kph = 0
start_point = (0,0)

pixels_per_meter = 10

while True:
    ret, frame1 = cap.read()

    # transform frame
    dilatada = transform_frame(frame1)
    
    # find contours
    contourShape, h = cv2.findContours(dilatada, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    #cv2.line(frame1, (5, count_line_position), (1900, count_line_position), (255,127,0), 1) # REMOVED
    
    center_points_cur_frame = []     # NEW ADD TESTING ADD
    
    for(i,c) in enumerate(contourShape):
        (x, y, w, h) = cv2.boundingRect(c)
        
        # check if contour is large enough
        contour_valid = (w >= min_contour_width) and (h >= min_contour_height)
        if not contour_valid:
            continue
                
        cv2.rectangle(frame1,(x,y),(x+w,y+h),(255,0,0),2)
        cv2.putText(frame1, "Vehicles" + str(counter), (x, y-20), cv2.FONT_HERSHEY_SIMPLEX, 1,(255, 244, 0), 2)
        
        centroid = get_centroid(x, y, w, h)
        
        center_points_cur_frame.append(centroid) # NEW ADD
                
        detect.append(centroid)
        """
        cv2.circle(frame1,centroid, 4, (0,255,0), -1)

        for (x, y) in detect:
            if y < (count_line_position + offset) and y > (count_line_position - offset):
                counter += 1
                cv2.line(frame1, (5, count_line_position), (700, count_line_position), (0, 127, 255), 1)
                detect.remove((x, y))
                print("Vehicle Count: " + str(counter))

                # calculate speed
                end_point = (x, y)
                speed_kph = calculate_speed(start_point, end_point, fps)
                print(f"Speed of vehicle {counter}: {speed_kph:.2f} km/h")
        """ 
        if counter <= 2: # NEW ADD -> only at beginning do we compare previous and current frame
            for centroid in center_points_cur_frame: # NEW ADD
                for centroid2 in center_points_prev_frame:
                    distance = math.hypot(centroid2[0] - centroid[0], centroid2[1] - centroid[1])

                    if distance < 20:
                        tracking_objects[track_id] = centroid
                        track_id += 1
        
        else:
            for centroid in center_points_cur_frame:
                for object_id, centroid2 in tracking_objects.items():
                    distance = math.hypot(centroid2[0] - centroid[0], centroid2[1] - centroid[1])
                    if distance < 20:
                        tracking_objects[object_id] = centroid
            
        
        for object_id, pt in tracking_objects.items(): # NEW ADD
            cv2.circle(frame1, pt, 4, (0,255,0), -1)
            cv2.putText(frame1, str(object_id), (pt[0], pt[1] - 7), 0, 1, (0, 0, 255), 1)
                    
            #cv2.circle(frame1, centroid, 4, (0,255,0), -1) # NEW ADD
            
        # make copy of points
        center_points_prev_frame = center_points_cur_frame.copy() # NEW ADD
         
    # Update fps and total frames
    total_frames += 1
    fps_end_time = time.time()
    fps = total_frames / (fps_end_time - fps_start_time)

    # Display speed and fps
    # cv2.putText(frame1, f"Speed: {speed_kph:.2f} km/h", (x, y - 50), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
    # cv2.putText(frame1, f"FPS: {fps:.2f}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
    
    cv2.imshow('original', frame1)
    
    # define keyboard break
    key = cv2.waitKey(1)
    if key == ord('q'):
        break
    
cv2.destroyAllWindows()
cap.release()
